In [7]:
import os
import sys

sys.path.append(os.path.join(os.path.abspath(""), os.pardir))

import datetime as dt
import pandas as pd
import requests
from prefect.blocks.system import Secret
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session
from mcpdb.tables import (
    Provider,
    ProviderAsset,
    ProviderContent,
    ProviderType,
)
from src.shared.utils import compare_dataframes

In [19]:
table_1 = pd.DataFrame(
    {
        "id": [1, 2, 3],
        "name": ["Alice", "Bob", "Charlie"],
    }
)

table_2 = pd.DataFrame(
    {
        "id": [1, 2, 3],
        "name": ["Alice", "Alex", "Charlie"],
    }
)

result = compare_dataframes(table_1, table_2, ["id"])
result

(Empty DataFrame
 Columns: [id, name_1, name_2, _merge]
 Index: [],
 Empty DataFrame
 Columns: [id, name_1, name_2, _merge]
 Index: [],
    id     name
 0   1    Alice
 2   3  Charlie,
    id  name
 1   2  Alex)

In [20]:
result[3]

,id,name
1,2,Alex


In [16]:
result[2]

,id,name
0,1,Alice
2,3,Charlie


In [17]:
result[3]

,id,name
1,2,Alex


In [ ]:
postgresql_password: str = (await Secret.load("postgresql-password")).get()  # type: ignore
host = (await Secret.load("postgresql-host")).get()  # type: ignore
port = 25060
database = "defaultdb"
user = "doadmin"
url = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user=user,
    password=postgresql_password,
    host=host,
    port=port,
    database=database,
)
engine = create_engine(url)

In [ ]:
categories: list[str] = []
with Session(engine) as session:
    # Get the coin desk provider.
    stmt = select(Provider).where(Provider.name == "CoinDesk")
    provider = session.execute(stmt).scalar_one_or_none()

    # Get all categories from provider assets.
    if provider:
        stmt = select(ProviderAsset).where(ProviderAsset.provider_id == provider.id)
        provider_assets = session.execute(stmt).scalars().all()
        for provider_asset in provider_assets:
            if provider_asset.asset_code:
                categories.append(provider_asset.asset_code)

categories

In [ ]:
host = "https://data-api.coindesk.com"
uri = "/news/v1/article/list"
params = {
    "lang": "EN",
    "limit": 100,
    "to_ts": (dt.datetime.now() - dt.timedelta(hours=2)).timestamp(),
    "categories": ",".join(categories),
}
response = requests.get(f"{host}{uri}", params=params)
if response.status_code != 200:
    raise Exception(f"Failed to fetch data from Coindesk API: {response.status_code}")
content_data = response.json()["Data"]
content_data

In [ ]:
host = "https://data-api.coindesk.com"
uri = "/news/v1/source/list"
params = {
    "lang": "EN",
    "status": "ACTIVE",
}
response = requests.get(f"{host}{uri}", params=params)
if response.status_code != 200:
    raise Exception(f"Failed to fetch data from Coindesk API: {response.status_code}")
provider_data = response.json()["Data"]
provider_data

In [ ]:
with Session(engine) as session:
    stmt = select(ProviderContent)
    provider_contents_df = pd.read_sql(stmt, engine)
provider_contents_df

In [ ]:
with Session(engine) as session:
    stmt = select(
        Provider.id,
        Provider.name,
        ProviderType.name.label("provider_type"),
        ProviderType.description.label("provider_type_description"),
        Provider.description,
        Provider.is_active,
        Provider.created_at,
        Provider.updated_at,
    ).join(ProviderType, Provider.provider_type_id == ProviderType.id)
    provider_df = pd.read_sql(stmt, engine)
provider_df

In [ ]:
with Session(engine) as session:
    stmt = select(ProviderType)
    provider_types_df = pd.read_sql(stmt, engine)
provider_types_df

In [ ]:
provider_data_df = pd.DataFrame(provider_data)
provider_data_df

In [ ]:
content_data_df = pd.DataFrame(content_data)
content_data_df

In [ ]:
url